In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
df = pd.read_csv('csv_flight/df_nums.csv')
df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportSeqID,Origin,OriginCityName,...,Diverted,CRSElapsedTime,Distance,is_holiday_week,OriginFlightDensity,Visibility,WindSpeed,DepDelay,delay_binary,delay_interval
0,2014,1,1,1,3,0,4974,1039705,0,0,...,0.0,141.0,689.0,1,2.0,10.0,5.0,-3.0,0,0
1,2014,1,1,1,3,1,1315,1039705,0,0,...,0.0,150.0,731.0,1,3.0,10.0,5.0,-2.0,0,0
2,2014,1,1,1,3,2,1134,1039705,0,0,...,0.0,128.0,606.0,1,3.0,10.0,5.0,2.0,1,1
3,2014,1,1,1,3,3,110,1039705,0,0,...,0.0,294.0,1947.0,1,6.0,10.0,3.0,21.0,1,3
4,2014,1,1,1,3,4,1347,1039705,0,0,...,0.0,323.0,2139.0,1,6.0,10.0,3.0,-2.0,0,0


In [3]:
df = df.drop(['Tail_Number', 'Flight_Number_Reporting_Airline'], axis=1)
df.dtypes

Year                     int64
Quarter                  int64
Month                    int64
DayofMonth               int64
DayOfWeek                int64
OriginAirportSeqID       int64
Origin                   int64
OriginCityName           int64
OriginState              int64
DestAirportSeqID         int64
Dest                     int64
DestCityName             int64
DestState                int64
CRSDepTime               int64
CRSArrTime               int64
Cancelled              float64
Diverted               float64
CRSElapsedTime         float64
Distance               float64
is_holiday_week          int64
OriginFlightDensity    float64
Visibility             float64
WindSpeed              float64
DepDelay               float64
delay_binary             int64
delay_interval           int64
dtype: object

#### Binary Classification

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
X = df.iloc[:, 0:23].values
scaler = StandardScaler()
X = scaler.fit_transform(X)

y = df.iloc[:, 24].values

# create 70% traning, 15% validation, 15% test split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=123)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=123)

X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val = torch.tensor(y_val, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

print("Shape of X_train:", X_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_val:", y_val.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: torch.Size([1147599, 23])
Shape of X_val: torch.Size([245914, 23])
Shape of X_test: torch.Size([245915, 23])
Shape of y_train: torch.Size([1147599])
Shape of y_val: torch.Size([245914])
Shape of y_test: torch.Size([245915])


In [6]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size=1, num_hidden_layers=10):
        super(SimpleNN, self).__init__()
        # first layer
        layers = [nn.Linear(input_size, hidden_size)]
        
        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_size, hidden_size))
        
        # output layer
        layers.append(nn.Linear(hidden_size, output_size))
        
        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        x = self.layers[-1](x)
        return x

In [7]:
model = SimpleNN(input_size=23, hidden_size=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
loss_function = nn.BCEWithLogitsLoss()

In [8]:
def train(model, train_features, train_labels, val_features, val_labels, optimizer, loss_function, epochs):
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        train_outputs = model(train_features)
        train_loss = loss_function(train_outputs.squeeze(), train_labels)
        train_predictions = torch.sigmoid(train_outputs).squeeze() > 0.5
        train_correct = (train_predictions == train_labels).sum().item()
        train_accuracy = train_correct / len(train_labels)
        train_loss.backward()
        optimizer.step()
        
        if epoch % 300 == 0:
            model.eval()
            with torch.no_grad():
                val_outputs = model(val_features)
                val_loss = loss_function(val_outputs.squeeze(), val_labels)
                val_predictions = torch.sigmoid(val_outputs).squeeze() > 0.5
                val_correct = (val_predictions == val_labels).sum().item()
                val_accuracy = val_correct / len(val_labels)
            print(f'Epoch {epoch+1}, Train Loss: {train_loss.item():.4f}, Train Accuracy: {train_accuracy * 100:.2f}%, '
                    f'Val Loss: {val_loss.item():.4f}, Val Accuracy: {val_accuracy * 100:.2f}%')

train(model, X_train, y_train, X_val, y_val, optimizer, loss_function, epochs=6000)

Epoch 1, Train Loss: 0.6903, Train Accuracy: 58.17%, Val Loss: 0.6901, Val Accuracy: 57.91%
Epoch 301, Train Loss: 0.6385, Train Accuracy: 62.96%, Val Loss: 0.6396, Val Accuracy: 62.76%
Epoch 601, Train Loss: 0.6310, Train Accuracy: 64.12%, Val Loss: 0.6329, Val Accuracy: 63.88%
Epoch 901, Train Loss: 0.6275, Train Accuracy: 64.60%, Val Loss: 0.6299, Val Accuracy: 64.33%
Epoch 1201, Train Loss: 0.6257, Train Accuracy: 64.86%, Val Loss: 0.6286, Val Accuracy: 64.52%
Epoch 1501, Train Loss: 0.6245, Train Accuracy: 64.98%, Val Loss: 0.6278, Val Accuracy: 64.58%
Epoch 1801, Train Loss: 0.6233, Train Accuracy: 65.13%, Val Loss: 0.6264, Val Accuracy: 64.74%
Epoch 2101, Train Loss: 0.6223, Train Accuracy: 65.24%, Val Loss: 0.6257, Val Accuracy: 64.81%
Epoch 2401, Train Loss: 0.6217, Train Accuracy: 65.31%, Val Loss: 0.6250, Val Accuracy: 64.97%
Epoch 2701, Train Loss: 0.6208, Train Accuracy: 65.45%, Val Loss: 0.6244, Val Accuracy: 65.06%
Epoch 3001, Train Loss: 0.6203, Train Accuracy: 65.50%, 

In [9]:
def test(model, test_features, test_labels, loss_function):
    model.eval() 
    with torch.no_grad():  
        test_outputs = model(test_features)
        test_loss = loss_function(test_outputs.squeeze(), test_labels)
        test_predictions = torch.sigmoid(test_outputs).squeeze() > 0.5
        test_correct = (test_predictions == test_labels).sum().item()
        test_accuracy = test_correct / len(test_labels)
        
    print(f'Test Loss: {test_loss.item():.4f}, Test Accuracy: {test_accuracy * 100:.2f}%')
    
test(model, X_test, y_test, loss_function)

Test Loss: 0.6222, Test Accuracy: 65.26%


#### Multiclass Classification

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [11]:
X = df.iloc[:, 0:23].values
scaler = StandardScaler()
X = scaler.fit_transform(X)

y = df.iloc[:, 25].values

# create 70% traning, 15% validation, 15% test split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=123)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=123)

X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.long).to(device)
X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val = torch.tensor(y_val, dtype=torch.long).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.long).to(device)

print("Shape of X_train:", X_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_val:", y_val.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: torch.Size([1147599, 23])
Shape of X_val: torch.Size([245914, 23])
Shape of X_test: torch.Size([245915, 23])
Shape of y_train: torch.Size([1147599])
Shape of y_val: torch.Size([245914])
Shape of y_test: torch.Size([245915])


In [12]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size=14, num_hidden_layers=10):
        super(SimpleNN, self).__init__()
        # first layer
        layers = [nn.Linear(input_size, hidden_size)]
        
        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_size, hidden_size))
        
        # output layer
        layers.append(nn.Linear(hidden_size, output_size))
        
        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        x = self.layers[-1](x)
        return x

In [13]:
model = SimpleNN(input_size=23, hidden_size=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
loss_function = nn.CrossEntropyLoss()

In [14]:
def train(model, train_features, train_labels, val_features, val_labels, optimizer, loss_function, epochs):
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        train_outputs = model(train_features)
        train_loss = loss_function(train_outputs, train_labels)
        _, train_predictions = torch.max(train_outputs, 1)
        train_correct = (train_predictions == train_labels).sum().item()
        train_accuracy = train_correct / len(train_labels)
        train_loss.backward()
        optimizer.step()
        
        if epoch % 300 == 0:
            model.eval()
            with torch.no_grad():
                val_outputs = model(val_features)
                val_loss = loss_function(val_outputs, val_labels)
                _, val_predictions = torch.max(val_outputs, 1)
                val_correct = (val_predictions == val_labels).sum().item()
                val_accuracy = val_correct / len(val_labels)
            print(f'Epoch {epoch+1}, Train Loss: {train_loss.item():.4f}, Train Accuracy: {train_accuracy * 100:.2f}%, '
                    f'Val Loss: {val_loss.item():.4f}, Val Accuracy: {val_accuracy * 100:.2f}%')

train(model, X_train, y_train, X_val, y_val, optimizer, loss_function, epochs=6000)

Epoch 1, Train Loss: 2.6525, Train Accuracy: 4.18%, Val Loss: 2.6493, Val Accuracy: 4.22%
Epoch 301, Train Loss: 1.3629, Train Accuracy: 58.17%, Val Loss: 1.3665, Val Accuracy: 57.91%
Epoch 601, Train Loss: 1.3530, Train Accuracy: 58.17%, Val Loss: 1.3572, Val Accuracy: 57.91%
Epoch 901, Train Loss: 1.3465, Train Accuracy: 58.17%, Val Loss: 1.3507, Val Accuracy: 57.91%
Epoch 1201, Train Loss: 1.3418, Train Accuracy: 58.18%, Val Loss: 1.3469, Val Accuracy: 57.92%
Epoch 1501, Train Loss: 1.3377, Train Accuracy: 58.18%, Val Loss: 1.3431, Val Accuracy: 57.92%
Epoch 1801, Train Loss: 1.3341, Train Accuracy: 58.19%, Val Loss: 1.3399, Val Accuracy: 57.93%
Epoch 2101, Train Loss: 1.3316, Train Accuracy: 58.19%, Val Loss: 1.3377, Val Accuracy: 57.93%
Epoch 2401, Train Loss: 1.3298, Train Accuracy: 58.20%, Val Loss: 1.3361, Val Accuracy: 57.94%
Epoch 2701, Train Loss: 1.3283, Train Accuracy: 58.20%, Val Loss: 1.3348, Val Accuracy: 57.94%
Epoch 3001, Train Loss: 1.3272, Train Accuracy: 58.21%, Va

In [15]:
def test(model, test_features, test_labels, loss_function):
    model.eval() 
    with torch.no_grad():  
        test_outputs = model(test_features)
        test_loss = loss_function(test_outputs, test_labels)
        _, test_predictions = torch.max(test_outputs, 1)
        test_correct = (test_predictions == test_labels).sum().item()
        test_accuracy = test_correct / len(test_labels)
        
    print(f'Test Loss: {test_loss.item():.4f}, Test Accuracy: {test_accuracy * 100:.2f}%')
    
test(model, X_test, y_test, loss_function)

Test Loss: 1.3256, Test Accuracy: 58.25%
